In [1]:
import os
import gc
import json
from pathlib import Path

import numpy as np

import src.load as load
import jsbeautifier
from src._operations import group_by
from src.deconvolution import Deconvolution
from tqdm import tqdm

from scipy.stats import pearsonr, entropy
from scipy.spatial.distance import jensenshannon

from sklearn.model_selection import train_test_split as tts
from sklearn.utils.validation import column_or_1d

<h2>Data</h2>

Three datasets were used in this study.

1. **Idiopathic Pulmonary Fibrosis (IPF)**: 96,301 cells and 4,443 highly variable genes. (https://www.science.org/doi/10.1126/sciadv.aba1983)
2. **Mouse Cortex (MC)**: 3,005 cells and 20,006 genes. After filtering genes expressed in at least 10 cells, we are left with 16,484 genes. (https://www.science.org/doi/10.1126/science.aaa1934)
3. **Human Cell Atlas (HCA)**: 84,363 cells and 2,968 highly variable genes. (https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02210-0)

These datasets can be downloaded as .h5ad files from https://drive.google.com/drive/folders/1fLt3QGI2XFIz-4ZjOwk9poFpmqq1Zcgt?usp=sharing.

Since IPF and HCA are large, we only work with highly variable genes to reduce the runtime of the methods.
The IPF h5ad file contains highly variable genes only, so we only need to set `high_var` to `True` for HCA.
`filter_genes` will be set to True for both MC and HCA.

Only for scGeneFit we also set `high_var=True` for MC. Without it, the algorithm will take too long to run across all random seeds and coverage factors.

The default method for selecting highly variable genes in Scanpy requires the data to be log-transformed (https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.highly_variable_genes.html). However, deconvolution works best in linear space. For this reason, high variable genes for HCA have been saved into `data/he-hv.txt` and loaded from there to simply the code.

Finally, we remove all classes with less than 50 cells. This leads to 33 classes for IPF and 75 classes for HCA (tissue/cell type pairs).

In [2]:
dataset = Path('HCA') # choose from IPF, HCA, MC

In [4]:
root = 'data' / dataset
os.makedirs(root / 'images', exist_ok=True)

# We don't log for cibersort
adata, key = load.dataset(
    str(dataset),
    filter_genes=False if str(dataset) == 'IPF' else True,
    normalize=True,
    high_var=True if str(dataset) == 'HCA' else False,
    #high_var=True # set to True for MC is running scGeneFit
)
adata = load.remove_low_count_ct(adata, key, 50)
print("Range:", adata.X.min(), adata.X.max())

7 unique cell types.
adata.shape=(3005, 20006)
Removed low count classes.
adata.shape=(3005, 5034)
7 celltype combinations.
Range: 0.0 3107.6062


Split the data into train and test. Form a signature matrix using train data and form a pseudo-mixture using test data.

In [5]:
def get_signature(adata, key, *, rs=42, verbose=False):
    x_train, x_test = tts(adata, random_state=rs, stratify=adata.obs[key], train_size=0.5)
    # Form the signature matrix by averaging per phenotype
    signature = np.asarray(group_by(x_train.X, x_train.obs[key]))
    mixture = np.array(x_test.X.mean(axis=0)).flatten()
    _, ground_truth = np.unique(x_test.obs[key], return_counts=True)
    ground_truth = ground_truth.astype(float) / x_test.shape[0]
    if verbose:
        print(f"Formed signature matrix with shape {signature.shape}")
        print(f"Max value in the signature matrix: {signature.max()}")
        print(f"Formed mixture with shape {mixture.shape}")
    del x_train
    del x_test
    gc.collect()
    return signature, mixture, ground_truth

Run deconvolution using $\nu$-SVR (CIBERSORT).

In [6]:
def deconv(
        signature,
        mixture,
        *,
        dict_key,
        ground_truth,
        feature_selector=None,
        selected=None,
        json_path=None,
        report=None,
    ):
    """Run CIBERSORT using the given signature matrix
    and mixture by using only features returned by
    feature selector.
    """
    if selected is None:
        assert feature_selector is not None
        selected = feature_selector.get_support(indices=True)
    dv = Deconvolution(verbose=False)
    deconvolved = dv.fit_predict(
        signature.T[selected],
        column_or_1d(mixture)[selected])
    
    if report is None:
        report = {}
    report[dict_key] = {}
    report[dict_key]['n_features'] = len(selected)
    report[dict_key]['phenotypes'] = signature.shape[0]
    report[dict_key]['n_features_in'] = signature.shape[1]
    report[dict_key]['JS'] = jensenshannon(deconvolved, ground_truth)
    report[dict_key]['entropy'] = entropy(deconvolved, ground_truth)
    report[dict_key]['pearson'] = pearsonr(deconvolved, ground_truth)[0]
    report[dict_key]['deconvolution'] = deconvolved.tolist()
    if isinstance(selected, np.ndarray):
        selected = selected.tolist()
    report[dict_key]['solution'] = selected
    # Dump json
    options = jsbeautifier.default_options()
    options.indent_size = 4
    beau_report = jsbeautifier.beautify(json.dumps(report), options)

    if json_path is not None:
        with open(json_path, "w") as f:
            f.write(beau_report)
    
    return report

def load_rep(path_to_json):
    with open(path_to_json, "r") as f:
        __report = json.load(f)
    return __report

<h2>Run deconvolution</h2>

In [7]:
report_filename = 'report.json'
deconv_filename = 'deconv.json'

base_dirs = [
    'GreedyCover',
    'DT',
    'TopDE',
    'ReliefF',
    'Fval',
    'MI',
    'mRMR',
    'CrossEntropy',
    'RankCorr',
    'scGeneFit',
]

In [ ]:
# For every random seed
for rs in range(42, 47):
    signature, mixture, ground_truth = get_signature(adata, key, rs=rs)
    sub_root = 'data' / dataset / 'RS' / f'rs{rs}'
    bd = [sub_root / m for m in base_dirs]
    
    # For every method
    for rd in tqdm(bd):
        f_report = load_rep(rd / report_filename)

        report = {}
        # For every coverage factor
        for cov in tqdm(f_report):
            _ = deconv(
                signature,
                mixture,
                dict_key=cov,
                ground_truth=ground_truth,
                json_path=rd / deconv_filename,
                selected=f_report[cov]['solution'],
                report=report,
        )